## **Hypothesis Testing**

This is a simple project to get data from a csv file and convert it into a table in postgresql, then use the table for upper tail hypothesis test.

The data is extracted from https://www.kaggle.com/datasets/justinas/nba-players-data?select=all_seasons.csv and adapted for our needs.

We want to see if our player will play for more than half of the games in the season or not. There are 90 games in a season, so we will use 45 as the mean value.

The null hypothesis adapted is H: u<=45 against H: u>45 with alpha level 0.05

In [3]:
import psycopg2
import pandas as pd
import scipy.stats as stats
import math


Connect to PostgreSQL on the local server

In [10]:
con = psycopg2.connect(
    host = "localhost",
    database = "players",
    port = "5432",
    user = "postgres",
    password = "1536"
)
con.autocommit = True
cur = con.cursor()

Import the dataset, and transfer it into a table in postgresql.

In [11]:
data =  pd.read_csv("playerstats.csv")
df = pd.DataFrame(data)
cur.execute("CREATE TABLE players (id serial PRIMARY KEY, name varchar, games integer)")

for row in df.itertuples():
    cur.execute("INSERT INTO players (name, games)  VALUES (%s,%s)", (row.plname, row.gp))

The functions below will obtain values necessary for hypothesis testing.

meanofgames() obtains the mean number of games played by all players.

stddev() obtains the standard deviation of the number of games played by all players.

samplesize() obtains the sample size of the number of players 

In [16]:
def meanofgames():
    cur.execute("SELECT AVG(GAMES) FROM PLAYERS;")
    for mean in cur.fetchone():
        return mean
    
def stddev():
    cur.execute("SELECT STDDEV(GAMES) FROM PLAYERS")
    for stddev in cur.fetchone():
        return stddev

def samplesize():
    cur.execute(" SELECT COUNT(GAMES) FROM PLAYERS")
    for size in cur.fetchone():
        return size

90 games are played in one season. We want to see the chances a player will play...We will define 45 as the mean value for our hypothesis test. 

As we are performing a one-tail test (upper tail), we define the z-value as 0.95

Then we obtain the critical value for the hypothesis test based on z-value, standard deviation, and sample size.

In [18]:
testmean = 45
zvalue = stats.norm.ppf(0.95)
x_crit = zvalue*int(stddev())/math.sqrt(samplesize()) +45

Lastly, we use an if-else statement to finalize the hypothesis test by comparing the mean number of games played by the players to the critical value and prints a message indicating whether to reject or fail to reject the null hypothesis. 

We finalize by closing the connection to database.

In [19]:
if meanofgames() > x_crit:
    print("Reject the null hypothesis")
else:
    print("There isnt sufficient evidence to reject the null hypothesis")



con.close()

There isnt sufficient evidence to reject the null hypothesis
